<a href="https://colab.research.google.com/github/Laasaa/Krypto/blob/main/Kryptohistoria_kuvaajaksi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# -*- coding: utf-8 -*-

# Kryptohistoria kuvaajaksi#
# -------------------------#


# Tuodaan cryptocompare moduuli kryptotietojen hakemista varten cryptocompare-sivustolta
!pip install cryptocompare
import cryptocompare

# Tuodaan pandas moduuli datan käsittelyyn
import pandas as pd

# Tuodaan datetime
from datetime import datetime

# Tuodaan matplot.pyplotista plt kuvaajien piirtämistä varten
import matplotlib.pyplot as plt

# Tuodaan plotlysta moduuleja kuvaajien piirtämistä varten
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Syötetyn aikadatan testaus
def test_date(coin, cur_pair, year, month=12, day=28):
    while True:
        try:
            raw_price_data = \
                cryptocompare.get_historical_price_day(coin, cur_pair, limit=1, exchange='CCCAGG',
                                                       toTs=datetime(year, month, day, 0, 0))

        # Jos käyttäjä syöttää liikaa päiviä kuukautta kohden
        except ValueError:
            print('Haetussa kuukaudessa ei ole noin monta päivää')
            break

        # Muutetaan raakamuotoinen hinta-aineisto pandas dataframeksi
        day_price_data = pd.DataFrame.from_dict(raw_price_data)

        # Tarkistetaan että dataa löytyy haetulta aikaväliltä
        if day_price_data.close[0] == 0 or len(day_price_data.close) <= 1:
          # Jos dataa ei löydy
            print('Dataa ei löydy haetulta ajalta')
            return 0
        else:
        # Jos dataa löytyy
            return 1

# Syötetyn datan pituuden testaus
def test_length(coin, cur_pair, year, month, day, length):
    while True:
        try:
            raw_price_data = \
                cryptocompare.get_historical_price_day(coin, cur_pair, limit=length+28, exchange='CCCAGG',
                                                       toTs=datetime(year, month, day, 0, 0))

        # Jos käyttäjä syöttää liikaa päiviä kuukautta kohden
        except ValueError:
            print('Haetussa kuukaudessa ei ole noin monta päivää')
            break

        # Muutetaan raakamuotoinen hinta-aineisto pandas dataframeksi
        day_price_data = pd.DataFrame.from_dict(raw_price_data)

        # Tarkistetaan että dataa löytyy haetulta aikaväliltä
        if day_price_data.close[0] == 0 or len(day_price_data.close) <= 1:
          # Jos hintahistoriaa ei löydy
            print('Hintahistoriaa ei löydy noin pitkälle aikavälille')
            return 0
        else:
          # Jos hintahistoriaa löytyy
            return 1

# RSI-alifunktio
def get_rsi(close, lookback):
    ret = close.diff()
    up = []
    down = []
    for i in range(len(ret)):
        if ret[i] < 0:
            up.append(0)
            down.append(ret[i])
        else:
            up.append(ret[i])
            down.append(0)
    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()
    up_ewm = up_series.ewm(com=lookback - 1, adjust=False).mean()
    down_ewm = down_series.ewm(com=lookback - 1, adjust=False).mean()
    rs = up_ewm / down_ewm
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns={0: 'rsi'}).set_index(close.index)
    rsi_df = rsi_df.dropna()
    return rsi_df[3:]


time = 0
chart = 0
coin = 'USD'
coin2 = 'USD'

# Pääfunktio
def main():
    while True:
        while True:
         try:
            # Valikko
            choice = int(input('\n(1)CSV-lista suurimmista kryptovaluutoista (11.1.2023)\n(2)Valuutan tämänhetkinen arvo euroina\n'
                                '(3)Yksinkertainen viivakartta\n(4)Kynttiläkuvaaja indikaattoreilla\n(5)Lopeta ohjelma\n'))
            if choice in range(1, 6):
                break
            else:
                print('Valitse joku vaihtoehdoista')
         except ValueError:
            print('Valitse joku vaihtoehdoista')

        # Luetaan CSV:stä kolikkojen tietoja
        if choice == 1:
            try:
                coinlist = pd.read_csv('coinlist.csv')
                print(coinlist)
                print('Tiedot haettu 11.1.2023')
            except FileNotFoundError:
                print('Tiedostoa ei löydy')

        # Hakee kolikon vaihtosuhteen euroina (Niitä on muuten listattuna tällä hetkellä 3241 kpl)
        elif choice == 2:
            val_nimi = str(input('Anna valuutan lyhenne (Esim. ETH,BTC,XRP): '))
            val_hinta = cryptocompare.get_price(val_nimi)
            if val_hinta == None:
                print('Kyseistä valuuttaa ei löytynyt')
            else:
                print('Vaihtosuhde on:', val_hinta)

        # Yksinkertainen viivakartta
        elif choice == 3:
            while True:
              # Kysytään valuutan lyhennettä
                coin = str(input('Anna valuutan lyhenne (Esim. ETH,BTC,XRP): '))
                got_coin = cryptocompare.get_price(coin)
                # Jos valuuttaa ei löydy
                if got_coin == None:
                    print('Kyseistä valuuttaa ei löytynyt')
                    continue
                else:
                    break

            while True:
              # Kysytään vertailtavaa valuuttaa
                cur_pair = str(input('Mihin valuuttaa verrataan? (Esim. USDT,USD,EUR,BTC)'))
                got_cur = cryptocompare.get_price(cur_pair)
                if got_cur == None:
                  # Jos valuuttaa ei löydy
                    print('Kyseistä valuuttaa ei löytynyt')
                    continue
                else:
                    break

            while True:
              # Kysytään vuotta
                try:
                    year = int(input('Vuosi: '))
                    # Haetaan nykyinen vuosi
                    currentYear = datetime.now().year
                    # Tarkistetaan, että haettu vuosi ei ole liian kaukana historiassa tai tulevaisuudessa
                    if year > 2013 and year <= currentYear:
                      # Kutsutaan aikadatan testaus alifuktiota
                        result = test_date(coin, cur_pair, year)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    else:
                        print('Haettu vuosi on liian kaukana, tai elät tulevaisuudessa')
                except ValueError:
                    print('Anna kokonaislukuja')

            while True:
                try:
                  # Kysytään kuukautta
                    month = int(input('Kuukausi: '))
                    currentMonth = datetime.now().month
                    # Tarkistetaan, että haettu kuukausi ei ole tulevaisuudessa
                    if year == currentYear and month > currentMonth:
                        print('Haettu kuukausi on tulevaisuudessa')
                    elif (month > 0 and month <= 12 and year < currentYear):
                      # Kutsutaan aikadatan testaus alifuktiota
                        result = test_date(coin, cur_pair, year, month)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    # Jos haettu kuukausi on nykyinen
                    elif year == currentYear and month == currentMonth:
                      # Kutsutaan aikadatan testaus alifuktiota
                        result = test_date(coin, cur_pair, year, month)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    else:
                        print('Haettu kuukausi on virheellinen')
                except ValueError:
                    print('Anna kokonaislukuja')

            while True:
                try:
                  # Kysytään päivää
                    day = int(input('Päivä: '))
                    currentDay = datetime.now().day
                     # Tarkistetaan, että haettu päivä ei ole tulevaisuudessa
                    if year == currentYear and day > currentDay:
                        print('Haettu päivä on tulevaisuudessa')
                    elif (day > 0 and day <= 31 and year < currentYear):
                      # Kutsutaan aikadatan testaus alifuktiota
                        result = test_date(coin, cur_pair, year, month, day)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    elif day > 0 and day <= currentDay and year == currentYear:
                      # Kutsutaan aikadatan testaus alifuktiota
                        result = test_date(coin, cur_pair, year, month, day)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    else:
                        print('Haettu päivä on virheellinen')
                except ValueError:
                    print('Anna kokonaislukuja')

            while True:
              # Kysytään haluaako käyttäjä toisen valuutan kuvaajaan
                other_coin = str(input('Toinen valuutta kuvaajaan? y/n'))
                if other_coin == 'y' or other_coin == 'n':
                    break
                else:
                    print('Anna oikea arvo')

            if other_coin == 'y':
                while True:
                  # Kysytään toisen valuutan lyhennettä
                    coin2 = str(input('Anna valuutan lyhenne (Esim. ETH,BTC,XRP): '))
                    got_coin2 = cryptocompare.get_price(coin2)
                    if got_coin2 == None:
                        print('Kyseistä valuuttaa ei löytynyt')
                        continue
                    else:
                        break

                while True:
                  # Kysytään datapointtien tiheyttä
                    time = str(input('Datapointtien tiheys? minuutti(vain 7 viimeisintä päivää), tunti, päivä): '))
                    if time == 'minuutti' or time == 'tunti' or time == 'päivä':
                        break
                    else:
                        print('Et antanut oikeaa arvoa')

                while True:
                    try:
                        # Kysytään kuinka pitkän historian käyttäjä haluaa
                        length = int(input('Kuinka pitkän historian haluat? (max 1950): '))
                        if length > 0 and length <= 1950:
                          # Kutsutaan historian pituuden testaus alifunktiota
                            result2 = test_length(coin, cur_pair, year, month, day, length)
                            if result2 == 0:
                                continue
                            elif result2 == 1:
                                break
                        else:
                            print('Virheellinen historian pituus')
                    except ValueError:
                        print('Anna kokonaislukuja')

                while True:
                        # Kysytään haluaako käyttäjä kurssin samaan vai eri kuvaajaan
                        chart_ask = str(input('Piirretäänkö kurssi samaan vai eri kuvaajaan? (samaan/eri)'))
                        if chart_ask == 'samaan':
                            chart = 1
                            break
                        if chart_ask == 'eri':
                            chart = 2
                            break
                        else:
                            print('Valitse oikea vaihtoehto')
            # Jos käyttäjä haluaa vain yhden valuutan
            if other_coin == 'n':
                chart = 0
                coin2 ='USD'
                while True:
                    time = str(
                        input('Datapointtien tiheys? minuutti(vain 7 viimeisintä päivää), tunti, päivä: '))
                    if time == 'minuutti' or time == 'tunti' or time == 'päivä':
                        break
                    else:
                        print('Et antanut oikeaa arvoa')

                while True:
                    try:
                        # Kysytään kuinka pitkän historian käyttäjä haluaa
                        length = int(input('Kuinka pitkän historian haluat? (max 1950): '))
                        if length > 0 and length <= 1950:
                            # Kutsutaan historian pituuden testaus alifunktiota
                            result2 = test_length(coin, cur_pair, year, month, day, length)
                            if result2 == 0:
                                continue
                            elif result2 == 1:
                                break
                        else:
                            print('Virheellinen historian pituus')
                    except ValueError:
                        print('Anna kokonaisluku')

            # Jos käyttäjä valitsee datantiheydeksi minuutin
            # Minuuttidata
            if time == 'minuutti':
                # Koetetaan hankkia raakamuotoinen hinta-aineisto
                try:
                    raw_price_data = \
                    cryptocompare.get_historical_price_minute(coin, cur_pair, limit=length+4, exchange='CCCAGG',
                                                           toTs=datetime(year, month, day, 0, 0))
                    raw_price_data2 = \
                    cryptocompare.get_historical_price_minute(coin2, cur_pair, limit=length+4, exchange='CCCAGG',
                                                           toTs=datetime(year, month, day, 0, 0))

                # Jos käyttäjä syöttää liikaa päiviä kuukautta kohden
                except ValueError:
                    print('Liikaa päiviä kuukaudessa')
                    break

                # Muutetaan raakamuotoinen hinta-aineisto pandas dataframeksi
                minute_price_data = pd.DataFrame.from_dict(raw_price_data)
                minute_price_data2 = pd.DataFrame.from_dict(raw_price_data2)


                # Muutetaan time datetime muotoon
                minute_price_data.set_index('time', inplace=True)
                minute_price_data.index = pd.to_datetime(minute_price_data.index, unit='s')
                minute_price_data2.set_index('time', inplace=True)
                minute_price_data2.index = pd.to_datetime(minute_price_data2.index, unit='s')

                # Lasketaan aineistolle RSI
                minute_price_data['rsi_14'] = get_rsi(minute_price_data['close'], 14)
                minute_price_data = minute_price_data.dropna()
                minute_price_data2['rsi_14'] = get_rsi(minute_price_data2['close'], 14)
                minute_price_data2 = minute_price_data2.dropna()

                # Määritellään kuvaajan attribuutit, jos halutaan vain yksi kuvaaja
                if chart == 0:
                    plt.figure(figsize=(15, 5))
                    plt.title(coin, fontsize=16)
                    plt.style.use('ggplot')
                    plt.xlabel('Y/D/H', fontsize=15)
                    plt.ylabel(cur_pair.upper(), fontsize=17)
                    plt.tick_params(axis='both', labelsize=15)
                    plt.plot(minute_price_data.close, label='Arvo', linestyle='-', color='green')
                    plt.legend(loc='best', prop={'size': 12})
                    plt.show()

                # Jos halutaan 2 kuvaajaa samaan karttaan
                if chart == 1:
                    plt.figure(figsize=(15, 5))
                    plt.title(coin.upper(), 'ja', coin2.upper(), fontsize=16)
                    plt.style.use('ggplot')
                    plt.xlabel('Y/D/H', fontsize=15)
                    plt.ylabel(cur_pair.upper(), fontsize=17)
                    plt.tick_params(axis='both', labelsize=15)
                    plt.plot(minute_price_data.close, label=coin.upper(), linestyle='-', color='green')
                    plt.plot(minute_price_data2.close, label=coin2.upper(), linestyle='-', color='red')
                    plt.legend(loc='best', prop={'size': 12})
                    plt.show()

                # Jos halutaan 2 kuvaajaa eri karttoihin
                if chart == 2:
                    fig, (ax1, ax2) = plt.subplots(2, figsize=(10, 10))
                    fig.tight_layout(pad=3.0)
                    ax1.set_title(coin, fontsize=12)
                    ax1.set_ylabel(cur_pair.upper())
                    ax1.plot(minute_price_data.close, label='Hintakäyrä', linestyle='-', color='green')
                    ax1.legend(loc='best', prop={'size': 10})

                    ax2.set_title(coin2, fontsize=12)
                    ax2.set_xlabel('time', fontsize=12)
                    ax2.set_ylabel(cur_pair.upper())
                    ax2.legend(loc='best', prop={'size': 12})
                    ax2.plot(minute_price_data2.close, label='Hintakäyrä', linestyle='-', color='green')
                    ax2.legend(loc='best', prop={'size': 10})
                    plt.show()

            # Jos käyttäjä valitsee datantiheydeksi tunnin
            # Tuntidata
            if time == 'tunti':
                try:
                    # Koetetaan hankkia raakamuotoinen hinta-aineisto
                    raw_price_data = \
                    cryptocompare.get_historical_price_hour(coin, cur_pair, limit=length+4, exchange='CCCAGG',
                                                           toTs=datetime(year, month, day, 0, 0))
                    raw_price_data2 = \
                    cryptocompare.get_historical_price_hour(coin2, cur_pair, limit=length+4, exchange='CCCAGG',
                                                           toTs=datetime(year, month, day, 0, 0))
                except ValueError:
                    print('Liikaa päiviä kuukaudessa')
                    break

                # Muutetaan raakamuotoinen hinta-aineisto pandas dataframeksi
                hourly_price_data = pd.DataFrame.from_dict(raw_price_data)
                hourly_price_data2 = pd.DataFrame.from_dict(raw_price_data2)

                # Muutetaan aika datetime muotoon
                hourly_price_data.set_index('time', inplace=True)
                hourly_price_data.index = pd.to_datetime(hourly_price_data.index, unit='s')
                hourly_price_data2.set_index('time', inplace=True)
                hourly_price_data2.index = pd.to_datetime(hourly_price_data2.index, unit='s')

                # Lasketaan aineistolle RSI
                hourly_price_data['rsi_14'] = get_rsi(hourly_price_data['close'], 14)
                hourly_price_data = hourly_price_data.dropna()
                hourly_price_data2['rsi_14'] = get_rsi(hourly_price_data2['close'], 14)
                hourly_price_data2 = hourly_price_data2.dropna()

                # Määritellään kuvaajan attribuutit, jos halutaan vain yksi kuvaaja
                if chart == 0:
                    plt.figure(figsize=(15, 5))
                    plt.title(coin, fontsize=16)
                    plt.style.use('ggplot')
                    plt.xlabel('Y/D/H', fontsize=15)
                    plt.ylabel(cur_pair.upper(), fontsize=17)
                    plt.tick_params(axis='both', labelsize=15)
                    plt.plot(hourly_price_data.close, label=coin.upper(), linestyle='-', color='green')
                    plt.legend(loc='best', prop={'size': 12})
                    plt.show()


                # Jos halutaan 2 kuvaajaa samaan karttaan
                if chart == 1:
                    plt.figure(figsize=(15, 5))
                    plt.title('Hintakurssit', fontsize=16)
                    plt.style.use('ggplot')
                    plt.xlabel('Y/D/H', fontsize=15)
                    plt.ylabel(cur_pair.upper(), fontsize=17)
                    plt.tick_params(axis='both', labelsize=15)
                    plt.plot(hourly_price_data.close, label=coin.upper(), linestyle='-', color='green')
                    plt.plot(hourly_price_data2.close, label=coin2.upper(), linestyle='-', color='red')
                    plt.legend(loc='best', prop={'size': 12})
                    plt.show()

                # Jos halutaan 2 kuvaajaa eri karttoihin
                if chart == 2:
                    fig,(ax1, ax2) = plt.subplots(2, figsize=(10, 10))
                    fig.tight_layout(pad=3.0)
                    ax1.set_title(coin, fontsize=12)
                    ax1.set_ylabel(cur_pair.upper())
                    ax1.plot(hourly_price_data.close, label=coin.upper(), linestyle='-', color='green')
                    ax1.legend(loc='best', prop={'size': 10})

                    ax2.set_title(coin2, fontsize=12)
                    ax2.set_xlabel('time', fontsize=12)
                    ax2.set_ylabel(cur_pair.upper())
                    ax2.legend(loc='best', prop={'size': 12})
                    ax2.plot(hourly_price_data2.close, label=coin2.upper(), linestyle='-', color='green')
                    ax2.legend(loc='best', prop={'size': 10})
                    plt.show()

            # Jos käyttäjä valitsee datatiheydeksi päivän
            # Päivädata
            if time == 'päivä':
                try:
                    # Koetetaan hankkia raakamuotoinen hinta-aineisto
                    raw_price_data = \
                    cryptocompare.get_historical_price_day(coin, cur_pair, limit=length+4, exchange='CCCAGG',
                                                           toTs=datetime(year, month, day, 0, 0))
                    raw_price_data2 = \
                    cryptocompare.get_historical_price_day(coin2, cur_pair, limit=length+4, exchange='CCCAGG',
                                                           toTs=datetime(year, month, day, 0, 0))
                except ValueError:
                    print('Liikaa päiviä kuukaudessa')
                    break

                # Muutetaan raakamuotoinen hinta-aineisto pandas dataframeksi
                day_price_data = pd.DataFrame.from_dict(raw_price_data)
                day_price_data2 = pd.DataFrame.from_dict(raw_price_data2)

                # Muutetaan aika datetime muotoon
                day_price_data.set_index('time', inplace=True)
                day_price_data.index = pd.to_datetime(day_price_data.index, unit='s')
                day_price_data2.set_index('time', inplace=True)
                day_price_data2.index = pd.to_datetime(day_price_data2.index, unit='s')

                # Lasketaan aineistolle RSI
                day_price_data['rsi_14'] = get_rsi(day_price_data['close'], 14)
                day_price_data = day_price_data.dropna()
                day_price_data2['rsi_14'] = get_rsi(day_price_data2['close'], 14)
                day_price_data2 = day_price_data2.dropna()

                # Määritellään kuvaajan attribuutit, jos halutaan vain yksi kuvaaja
                if chart == 0:
                    plt.figure(figsize=(15, 5))
                    plt.title(coin, fontsize=16)
                    plt.style.use('ggplot')
                    plt.xlabel('Y/D/H', fontsize=15)
                    plt.ylabel(cur_pair.upper(), fontsize=17)
                    plt.tick_params(axis='both', labelsize=15)
                    plt.plot(day_price_data.close, label=coin.upper(), linestyle='-', color='green')
                    plt.legend(loc='best', prop={'size': 12})
                    plt.show()

                # Jos halutaan 2 kuvaajaa samaan karttaan
                if chart == 1:
                    plt.figure(figsize=(15, 5))
                    plt.title('Hintakurssit', fontsize=16)
                    plt.style.use('ggplot')
                    plt.xlabel('Y/D/H', fontsize=15)
                    plt.ylabel(cur_pair.upper(), fontsize=17)
                    plt.tick_params(axis='both', labelsize=15)
                    plt.plot(day_price_data.close, label=coin.upper(), linestyle='-', color='green')
                    plt.plot(day_price_data2.close, label=coin2.upper(), linestyle='-', color='red')
                    plt.legend(loc='best', prop={'size': 12})
                    plt.show()


                # Jos halutaan 2 kuvaajaa eri karttoihin
                if chart == 2:
                    fig,(ax1, ax2) = plt.subplots(2, figsize=(10, 10))
                    fig.tight_layout(pad=3.0)
                    ax1.set_title(coin, fontsize=12)
                    ax1.set_ylabel(cur_pair.upper())
                    ax1.plot(day_price_data.close, label=coin.upper(), linestyle='-', color='green')
                    ax1.legend(loc='best', prop={'size': 10})

                    ax2.set_title(coin2, fontsize=12)
                    ax2.set_xlabel('time', fontsize=12)
                    ax2.set_ylabel(cur_pair.upper())
                    ax2.legend(loc='best', prop={'size': 12})
                    ax2.plot(day_price_data2.close, label=coin2.upper(), linestyle='-', color='green')
                    ax2.legend(loc='best', prop={'size': 10})
                    plt.show()


        # Kynttiläkuvaaja indikaattoreilla
        elif choice == 4:
            while True:
                    # Kysytään valuutan lyhennettä
                    coin = str(input('Anna valuutan lyhenne (Esim. ETH,BTC,XRP): '))
                    got_coin = cryptocompare.get_price(coin)
                    if got_coin == None:
                        print('Kyseistä valuuttaa ei löytynyt')
                        continue
                    else:
                        break
            while True:
              # Kysytään vertailtavavan valuutan lyhennettä
                cur_pair = str(input('Mihin valuuttaa verrataan? (Esim. USDT,USD,EUR,BTC)'))
                got_cur = cryptocompare.get_price(cur_pair)
                if got_cur == None:
                    print('Kyseistä valuuttaa ei löytynyt')
                    continue
                else:
                    break

            while True:
                try:
                    # Kysytään vuotta
                    year = int(input('Vuosi: '))
                    currentYear = datetime.now().year
                    if year > 2013 and year <= currentYear:
                        result = test_date(coin, cur_pair, year)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    else:
                        print('Haettu vuosi on liian kaukana, tai elät tulevaisuudessa')
                except ValueError:
                    print('Anna kokonaislukuja')

            while True:
                try:
                    # Kysytään kuukautta
                    month = int(input('Kuukausi: '))
                    currentMonth = datetime.now().month
                    if year == currentYear and month > currentMonth:
                        print('Haettu kuukausi on tulevaisuudessa')
                    elif (month > 0 and month <= 12 and year < currentYear):
                        result = test_date(coin, cur_pair, year, month)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    elif year == currentYear and month == currentMonth:
                        result = test_date(coin, cur_pair, year, month)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    else:
                        print('Haettu kuukausi on virheellinen')
                except ValueError:
                    print('Anna kokonaislukuja')

            while True:
                try:
                    # Kysytään päivää
                    day = int(input('Päivä: '))
                    currentDay = datetime.now().day
                    if year == currentYear and day > currentDay:
                        print('Haettu päivä on tulevaisuudessa')
                    elif day > 0 and day <= 31 and year < currentYear:
                        result = test_date(coin, cur_pair, year, month, day)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    elif day > 0 and day <= currentDay and year == currentYear:
                        result = test_date(coin, cur_pair, year, month, day)
                        if result == 0:
                            continue
                        elif result == 1:
                            break
                    else:
                        print('Haettu päivä on virheellinen')
                except ValueError:
                    print('Anna kokonaislukuja')


            while True:
                    # Kysytään datapointtien tiheyttä
                    time = str(input('Datapointtien tiheys (minuutti(vain 7 viimeisintä päivää), tunti, päivä): '))
                    if time == 'minuutti' or time == 'tunti' or time == 'päivä':
                        break
                    else:
                        print('Et antanut oikeaa arvoa')

            while True:
                try:
                    # Kysytään historian pituutta
                    length = int(input('Kuinka pitkän historian haluat? (max 1950): '))
                    if length > 0 and length <= 1950:
                        result2 = test_length(coin, cur_pair, year, month, day, length)
                        if result2 == 0:
                            continue
                        elif result2 == 1:
                            break
                    else:
                        print('Virheellinen historian pituus')
                except ValueError:
                    print('Anna kokonaislukuja')

            # Jos käyttäjä valitsee datatiheydeksi minuutin
            # Minuuttidata
            if time == 'minuutti':
                # Koetetaan hankkia raakamuotoinen hinta-aineisto
                try:
                    raw_price_data = \
                        cryptocompare.get_historical_price_minute(coin, cur_pair, limit=length+3, exchange='CCCAGG', toTs=datetime(year, month, day, 0, 0))

                # Jos käyttäjä syöttää liikaa päiviä kuukautta kohden
                except ValueError:
                    print('Liikaa päiviä kuukaudessa')
                    break

                # Haetaan toinen hinta-aineisto Bollingerin nauhojen piirtämistä varten
                raw_price_data2 = \
                    cryptocompare.get_historical_price_minute(coin, cur_pair, limit=length+28, exchange='CCCAGG',
                                                            toTs=datetime(year, month, day, 0, 0))
                    
                # Muutetaan raakamuotoinen hinta-aineisto pandas dataframeksi
                minute_price_data = pd.DataFrame.from_dict(raw_price_data)
                minute_price_data2 = pd.DataFrame.from_dict(raw_price_data2)

                # Muutetaan time datetime muotoon
                minute_price_data.set_index('time', inplace=True)
                minute_price_data.index = pd.to_datetime(minute_price_data.index, unit='s')
                minute_price_data2.set_index('time', inplace=True)
                minute_price_data2.index = pd.to_datetime(minute_price_data2.index, unit='s')

                # Lasketaan aineistolle RSI-14
                minute_price_data['rsi_14'] = get_rsi(minute_price_data['close'], 14)
                minute_price_data = minute_price_data.dropna()

                # Lasketaan yksinkertainen liikkuva keskiarvo ja keskihajonta datalle
                window = 30
                minute_price_data2['sma'] = minute_price_data2['close'].rolling(window).mean()
                minute_price_data2['std'] = minute_price_data2['close'].rolling(window).std(ddof=0)

                # Poistetaan datasta ylimääräiset Bollingerin nauhojen piirtoon tarvitut rivit, muutetaan data CSV:ksi ja luetaan se
                minute_price_data2 = minute_price_data2.iloc[29:]
                minute_price_data.to_csv('coin.csv')
                minute_price_data = pd.read_csv('coin.csv')
                minute_price_data2.to_csv('coin2.csv')
                minute_price_data2 = pd.read_csv('coin2.csv')

                # Tehdään 3 erillistä kuvaajaa, joihin tulokset piirretään
                fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=(coin.upper(), 'RSI-14', 'Vaihtomäärä ($)'), vertical_spacing=0.1, row_width=[0.2, 0.2, 0.7])

                # Piirretään kynttiläkuvaaja
                fig.add_trace(go.Candlestick(x=minute_price_data['time'],
                                   open=minute_price_data['open'],
                                   high=minute_price_data['high'],
                                   low=minute_price_data['low'],
                                   close=minute_price_data['close'],
                                   showlegend=False,
                                   name='candlestick'),
                                   row=1, col=1)
                fig.update_layout(yaxis_title=cur_pair.upper())

                # Piirretään liukuva keskiarvo
                fig.add_trace(go.Scatter(x=minute_price_data2['time'],
                                         y=minute_price_data2['sma'],
                                         line_color='red',
                                         name='Yksinkertainen liukuva keskiarvo'),
                                         row=1, col=1)

                # Piirretään Bollingerin nauhan ylempi raja
                fig.add_trace(go.Scatter(x=minute_price_data2['time'],
                                         y=minute_price_data2['sma'] + (minute_price_data2['std'] * 2),
                                         line_color='gray',
                                         line={'dash': 'dash'},
                                         name='Bollingerin ylempi nauha',
                                         opacity=0.6),
                                         row=1, col=1)

                # Piirretään Bollingerin nauhan alempi raja ja täytetään nauhojen välinen alue
                fig.add_trace(go.Scatter(x=minute_price_data2['time'],
                                         y=minute_price_data2['sma'] - (minute_price_data2['std'] * 2),
                                         line_color='black',
                                         line={'dash': 'dash'},
                                         fill='tonexty',
                                         name='Bollingerin alempi nauha',
                                         opacity=0.6),
                                         row=1, col=1)

                # Piirretään RSI omaan kuvaajaansa
                fig.add_trace(go.Scatter(x=minute_price_data['time'],
                                         y=minute_price_data['rsi_14'],
                                         showlegend=False),
                                         row=2, col=1)
                # Piirretään volume omaan kuvaajaansa
                fig.add_trace(go.Bar(x=minute_price_data['time'],
                                     y=minute_price_data['volumeto'],
                                     marker_color='green',
                                     showlegend=False),
                                     row=3, col=1)

                # Poistetaan range-slider
                fig.update(layout_xaxis_rangeslider_visible=False)

                # Piirretään RSI-kuvaajaan y-akselin arvoille 70 ja 30 viiva
                fig.update_layout(
                    shapes=[
                        dict(type='line', xref='x', yref='y2',
                             x0=minute_price_data.time[0], y0=30,
                             x1=minute_price_data.time[length - 1],
                             y1=30, line_width=2, line_color='green'),
                        dict(type='line', xref='x', yref='y2',
                             x0=minute_price_data.time[0], y0=70,
                             x1=minute_price_data.time[length - 1],
                             y1=70, line_width=2, line_color='red')])

                fig.show()

            # Jos käyttäjä valitsee datatiheydeksi tunnin
            # Tuntidata
            if time == 'tunti':
                # Koetetaan hankkia raakamuotoinen hinta-aineisto
                try:
                    raw_price_data = \
                        cryptocompare.get_historical_price_hour(coin, cur_pair, limit=length+3, exchange='CCCAGG', toTs=datetime(year, month, day, 0, 0))

                # Jos käyttäjä syöttää liikaa päiviä kuukautta kohden
                except ValueError:
                    print('Liikaa päiviä kuukaudessa')
                    break

                # Haetaan toinen hinta-aineisto Bollingerin nauhojen piirtämistä varten
                raw_price_data2 = \
                    cryptocompare.get_historical_price_hour(coin, cur_pair, limit=length+28, exchange='CCCAGG',
                                                           toTs=datetime(year, month, day, 0, 0))

                # Muutetaan raakamuotoinen hinta-aineisto pandas dataframeksi

                hour_price_data = pd.DataFrame.from_dict(raw_price_data)
                hour_price_data2 = pd.DataFrame.from_dict(raw_price_data2)

                # Muutetaan time datetime muotoon
                hour_price_data.set_index('time', inplace=True)
                hour_price_data.index = pd.to_datetime(hour_price_data.index, unit='s')
                hour_price_data2.set_index('time', inplace=True)
                hour_price_data2.index = pd.to_datetime(hour_price_data2.index, unit='s')

                # Lasketaan aineistolle RSI-14
                hour_price_data['rsi_14'] = get_rsi(hour_price_data['close'], 14)
                hour_price_data = hour_price_data.dropna()

                # Lasketaan yksinkertainen liikkuva keskiarvo ja keskihajonta datalle
                window = 30
                hour_price_data2['sma'] = hour_price_data2['close'].rolling(window).mean()
                hour_price_data2['std'] = hour_price_data2['close'].rolling(window).std(ddof=0)

                # Poistetaan datasta ylimääräiset Bollingerin nauhojen piirtoon tarvitut rivit, muutetaan data CSV:ksi ja luetaan se

                hour_price_data2 = hour_price_data2.iloc[29:]
                hour_price_data.to_csv('coin.csv')
                hour_price_data = pd.read_csv('coin.csv')
                hour_price_data2.to_csv('coin2.csv')
                hour_price_data2 = pd.read_csv('coin2.csv')

                # Tehdään 3 erillistä kuvaajaa, joihin tulokset piirretään
                fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                                    subplot_titles=(coin.upper(), 'RSI-14', 'Vaihtomäärä ($)'),
                                    vertical_spacing=0.1,
                                    row_width=[0.2, 0.2, 0.7])

                # Piirretään kynttiläkuvaaja
                fig.add_trace(go.Candlestick(x=hour_price_data['time'],
                                             open=hour_price_data['open'],
                                             high=hour_price_data['high'],
                                             low=hour_price_data['low'],
                                             close=hour_price_data['close'],
                                             showlegend=False,
                                             name='candlestick'),
                                             row=1, col=1)
                fig.update_layout(yaxis_title=cur_pair.upper())

                # Piirretään liukuva keskiarvo
                fig.add_trace(go.Scatter(x=hour_price_data2['time'],
                                         y=hour_price_data2['sma'],
                                         line_color='red',
                                         name='Yksinkertainen liukuva keskiarvo'),
                                         row=1, col=1)

                # Piirretään Bollingerin nauhan ylempi raja
                fig.add_trace(go.Scatter(x=hour_price_data2['time'],
                                         y=hour_price_data2['sma'] + (hour_price_data2['std'] * 2),
                                         line_color='gray',
                                         line={'dash': 'dash'},
                                         name='Bollingerin ylempi nauha',
                                         opacity=0.5),
                                         row=1, col=1)

                # Piirretään Bollingerin nauhan alempi raja ja täytetään nauhojen välinen alue
                fig.add_trace(go.Scatter(x=hour_price_data2['time'],
                                         y=hour_price_data2['sma'] - (hour_price_data2['std'] * 2),
                                         line_color='black',
                                         line={'dash': 'dash'},
                                         fill='tonexty',
                                         name='Bollingerin alempi nauha',
                                         opacity=0.5),
                                         row=1, col=1)

                # Piirretään RSI omaan kuvaajaansa
                fig.add_trace(
                    go.Scatter(x=hour_price_data['time'],
                               y=hour_price_data['rsi_14'],
                               name='Relative strength index',
                               showlegend=False),
                               row=2, col=1)

                # Piirretään vaihtomäärä omaan kuvaajaansa
                fig.add_trace(
                    go.Bar(x=hour_price_data['time'],
                           y=hour_price_data['volumeto'],
                           marker_color='green',
                           showlegend=False),
                           row=3, col=1)

                # Poistetaan range-slider
                fig.update(layout_xaxis_rangeslider_visible=False)

                # Piirretään RSI-kuvaajaan y-akselin arvoille 70 ja 30 viiva
                fig.update_layout(
                    shapes=[
                        dict(type='line', xref='x', yref='y2',
                             x0=hour_price_data.time[0], y0=30,
                             x1=hour_price_data.time[length - 1], y1=30,
                             line_width=2, line_color='green'),
                        dict(type='line', xref='x', yref='y2',
                             x0=hour_price_data.time[0], y0=70,
                             x1=hour_price_data.time[length - 1], y1=70,
                             line_width=2, line_color='red')])

                fig.show()

            # Jos käyttäjä valitsee datatiheydeksi päivän
            # Päivädata
            if time == 'päivä':
                # Koetetaan hankkia raakamuotoinen hinta-aineisto
                try:
                    raw_price_data = \
                        cryptocompare.get_historical_price_day(coin, cur_pair, limit=length+3, exchange='CCCAGG', toTs=datetime(year, month, day, 0, 0))

                # Jos käyttäjä syöttää liikaa päiviä kuukautta kohden
                except ValueError:
                    print('Liikaa päiviä kuukaudessa')
                    break

                # Haetaan toinen hinta-aineisto Bollingerin nauhojen piirtämistä varten
                raw_price_data2 = \
                    cryptocompare.get_historical_price_day(coin, cur_pair,limit=length+28, exchange='CCCAGG', toTs=datetime(year, month, day, 0, 0))

                # Muutetaan raakamuotoinen hinta-aineisto pandas dataframeksi
                day_price_data = pd.DataFrame.from_dict(raw_price_data)
                day_price_data2 = pd.DataFrame.from_dict(raw_price_data2)

                # Muutetaan time datetime muotoon
                day_price_data.set_index('time', inplace=True)
                day_price_data.index = pd.to_datetime(day_price_data.index, unit='s')

                day_price_data2.set_index('time', inplace=True)
                day_price_data2.index = pd.to_datetime(day_price_data2.index, unit='s')

                # Lasketaan aineistolle RSI-14
                day_price_data['rsi_14'] = get_rsi(day_price_data['close'], 14)
                day_price_data = day_price_data.dropna()


                # Lasketaan yksinkertainen liikkuva keskiarvo ja keskihajonta datalle
                window = 30
                day_price_data2['sma'] = day_price_data2['close'].rolling(window).mean()
                day_price_data2['std'] = day_price_data2['close'].rolling(window).std(ddof=0)

                # Poistetaan datasta ylimääräiset Bollingerin nauhojen piirtoon tarvitut rivit, muutetaan data CSV:ksi ja luetaan se
                day_price_data2 = day_price_data2.iloc[29:]
                day_price_data.to_csv('coin.csv')
                day_price_data = pd.read_csv('coin.csv')
                day_price_data2.to_csv('coin2.csv')
                day_price_data2 = pd.read_csv('coin2.csv')

                # Tehdään 3 erillistä kuvaajaa, joihin tulokset piirretään
                fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                                    subplot_titles=(coin.upper(), 'RSI-14', 'Vaihtomäärä ($)'),
                                    vertical_spacing=0.1,
                                    row_width=[0.2, 0.2, 0.7])

                # Piirretään kynttiläkuvaaja
                fig.add_trace(go.Candlestick(x=day_price_data['time'],
                                             open=day_price_data['open'],
                                             high=day_price_data['high'],
                                             low=day_price_data['low'],
                                             close=day_price_data['close'],
                                             showlegend=False,
                                             name='candlestick'),
                                             row=1, col=1)
                fig.update_layout(yaxis_title=cur_pair.upper())

                # Piirretään liukuva keskiarvo
                fig.add_trace(go.Scatter(x=day_price_data2['time'],
                                         y=day_price_data2['sma'],
                                         line_color='red',
                                         name='Yksinkertainen liukuva keskiarvo'),
                                         row=1, col=1)

                # Piirretään Bollingerin nauhan ylempi raja
                fig.add_trace(go.Scatter(x=day_price_data2['time'],
                                         y=day_price_data2['sma'] + (day_price_data2['std'] * 2),
                                         line_color='gray',
                                         line={'dash': 'dash'},
                                         name='Bollingerin ylempi nauha',
                                         opacity=0.4),
                                         row=1, col=1)

                # Piirretään Bollingerin nauhan alempi raja ja täytetään nauhojen välinen alue
                fig.add_trace(go.Scatter(x=day_price_data2['time'],
                                         y=day_price_data2['sma'] - (day_price_data2['std'] * 2),
                                         line_color='black',
                                         line={'dash': 'dash'},
                                         fill='tonexty',
                                         name='Bollingerin alempi nauha',
                                         opacity=0.4),
                                         row=1, col=1)

                # Piirretään RSI omaan kuvaajaansa
                fig.add_trace(go.Scatter(x=day_price_data['time'],
                                         y=day_price_data['rsi_14'],
                                         name='Relative strength index'),
                                         row=2, col=1)

                # Piirretään vaihtomäärä omaan kuvaajaansa
                fig.add_trace(go.Bar(x=day_price_data['time'],
                                     y=day_price_data['volumeto'],
                                     marker_color='green',
                                     showlegend=False),
                                     row=3, col=1)
                
                # Poistetaan range-slider
                fig.update(layout_xaxis_rangeslider_visible=False)

                # Piirretään RSI-kuvaajaan y-akselin arvoille 70 ja 30 viiva
                fig.update_layout(
                    shapes=[
                        dict(type='line', xref='x', yref='y2',
                             x0=day_price_data.time[0], y0=30,
                             x1=day_price_data.time[length-1], y1=30,
                             line_width=2, line_color='green',
                             name='Yliostettu'),
                        dict(type='line', xref='x', yref='y2',
                             x0=day_price_data.time[0], y0=70,
                             x1=day_price_data.time[length-1], y1=70,
                             line_width=2, line_color='red',
                             name='Ylimyyty')])

                # Piirretään kuvaaja
                fig.show()

        # Lopetus
        elif choice == 5:
            print('Ohjelma lopetetaan')
            break
        else:
            print('Valitse joku vaihtoehdoista')

if __name__ == '__main__':
    main()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


KeyboardInterrupt: ignored